In [12]:
from urllib.request import urlopen
import requests as req
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By

In [13]:
def get_book_image(no, url):
    with urlopen(url) as f:
        with open('./images/book/book' + str(no) +'.jpg','wb') as h:
            data = f.read()
            h.write(data)

In [16]:
# requests는 막혀있으므로 selenium을 이용함
def get_book_detail(no, book_url):
    
    # 브라우저를 열지않고 실행하기
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    
    browser = webdriver.Chrome(options=options)
    #browser = webdriver.Chrome()
    browser.get(book_url)
    
    reg = browser.page_source
    soup = bs(reg, 'html.parser')
    
    title = soup.find('span', 'prod_title')
    if title != None:
        title = title.text
        price = soup.find('span', 'price').find('span', 'val').text
        author = soup.select_one('div.author > a').text
        date = soup.select('table.tbl_row > tbody > tr')
        if len(date) > 0:
            date = date[1].find('td').text.strip()
        else:
            date = ''
            
        # 책이미지 저장하기
        src = soup.select_one('div.portrait_img_box.portrait > img')['src']
        if src != None:
            get_book_image(no, src)
        
        # 매장별 재고 가져오기
        regions = soup.select('div.tbl_col_wrap')
        stocks = {}
        for region in regions:
            stores = region.select('th')
            counts = region.select('span.text')
            for i in range(min(len(stores), len(counts))):
                store = stores[i].text
                count = int(counts[i].text)
                if store in ['평촌', '판교', '강남']:
                    stocks[store] = count
        print('제목:', title)
        print('   링크:', book_url)
        print('   가격:', price)
        print('   저자:', author)
        print('   발행일:', date)
        print('   재고:', stocks)

    browser.quit()

In [17]:
url = 'https://search.kyobobook.co.kr/search?gbCode=TOT&target=total&keyword='
key_word = '크롤링'
res = req.get(url + key_word)
soup = bs(res.text, 'html.parser')

books = soup.select('ul.prod_list > li.prod_item')

for i in range(len(books)):
    book_url = books[i].find('a', 'prod_info').get('href')
    print(i + 1, end = '. ')
    get_book_detail(i+1, book_url)
    print('-----'*10)
    
    if i >= 4: break

print('도서 조회가 끝났습니다.')

1. 제목: R을 이용한 웹 크롤링과 텍스트 분석
   링크: https://product.kyobobook.co.kr/detail/S000001940404
   가격: 22,000원
   저자: 권상희
   발행일: 2020년 08월 30일
   재고: {'강남': 1, '판교': 0, '평촌': 0}
--------------------------------------------------
2. 제목: 한입에 웹 크롤링
   링크: https://product.kyobobook.co.kr/detail/S000001890854
   가격: 23,400원
   저자: 김경록
   발행일: 2018년 09월 28일
   재고: {'강남': 1, '판교': 1, '평촌': 0}
--------------------------------------------------
3. 제목: 웹 크롤링 & 데이터 분석 with 파이썬
   링크: https://product.kyobobook.co.kr/detail/S000001033130
   가격: 18,000원
   저자: 장철원
   발행일: 2022년 01월 24일
   재고: {'강남': 1, '판교': 2, '평촌': 1}
--------------------------------------------------
4. 제목: 파이썬을 이용한 퀀트 투자 포트폴리오 만들기
   링크: https://product.kyobobook.co.kr/detail/S000200883665
   가격: 27,000원
   저자: 이현열
   발행일: 2023년 02월 17일
   재고: {'강남': 9, '판교': 1, '평촌': 1}
--------------------------------------------------
5. 제목: 파이썬과 40개의 작품들
   링크: https://product.kyobobook.co.kr/detail/S000001870248
   가격: 16,920원
   저자: 장문철
   발행일: 2